निम्नलिखित नोटबुक्स चलाने के लिए, यदि आपने अभी तक नहीं किया है, तो आपको एक मॉडल डिप्लॉय करना होगा जो `text-embedding-ada-002` को बेस मॉडल के रूप में उपयोग करता है और .env फाइल में डिप्लॉयमेंट नाम को `AZURE_OPENAI_EMBEDDINGS_ENDPOINT` के रूप में सेट करना होगा।


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

अगला, हम एम्बेडिंग इंडेक्स को एक पांडा डेटाफ़्रेम में लोड करने जा रहे हैं। एम्बेडिंग इंडेक्स एक JSON फ़ाइल में संग्रहित है जिसका नाम `embedding_index_3m.json` है। एम्बेडिंग इंडेक्स में प्रत्येक YouTube ट्रांसक्रिप्ट के लिए एम्बेडिंग्स शामिल हैं, जो अक्टूबर 2023 के अंत तक के हैं।


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

अब हम एक फ़ंक्शन बनाएंगे जिसका नाम `get_videos` होगा, जो क्वेरी के लिए Embedding Index में खोज करेगा। यह फ़ंक्शन क्वेरी से सबसे ज़्यादा मिलते-जुलते टॉप 5 वीडियो लौटाएगा। यह फ़ंक्शन इस तरह काम करता है:

1. सबसे पहले, Embedding Index की एक कॉपी बनाई जाती है।
2. इसके बाद, क्वेरी के लिए Embedding को OpenAI Embedding API की मदद से निकाला जाता है।
3. फिर Embedding Index में एक नया कॉलम बनाया जाता है जिसका नाम `similarity` है। इस कॉलम में क्वेरी Embedding और हर वीडियो सेगमेंट के Embedding के बीच की कोसाइन सिमिलैरिटी होती है।
4. इसके बाद, Embedding Index को `similarity` कॉलम के आधार पर फ़िल्टर किया जाता है। इसमें सिर्फ़ वही वीडियो शामिल किए जाते हैं जिनकी कोसाइन सिमिलैरिटी 0.75 या उससे ज़्यादा है।
5. अंत में, Embedding Index को `similarity` कॉलम के अनुसार सॉर्ट किया जाता है और टॉप 5 वीडियो लौटाए जाते हैं।


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

यह फ़ंक्शन बहुत सरल है, यह बस खोज क्वेरी के परिणामों को प्रिंट करता है।


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. सबसे पहले, Embedding Index को एक Pandas Dataframe में लोड किया जाता है।
2. इसके बाद, उपयोगकर्ता से एक क्वेरी दर्ज करने के लिए कहा जाता है।
3. फिर `get_videos` फंक्शन को क्वेरी के लिए Embedding Index में खोज करने के लिए कॉल किया जाता है।
4. अंत में, `display_results` फंक्शन को परिणाम उपयोगकर्ता को दिखाने के लिए कॉल किया जाता है।
5. इसके बाद उपयोगकर्ता से एक और क्वेरी दर्ज करने के लिए कहा जाता है। यह प्रक्रिया तब तक चलती रहती है जब तक उपयोगकर्ता `exit` नहीं लिखता।

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.hi.png)

आपसे एक क्वेरी दर्ज करने के लिए कहा जाएगा। कोई क्वेरी लिखें और एंटर दबाएँ। एप्लिकेशन उस क्वेरी से संबंधित वीडियो की एक सूची लौटाएगा। एप्लिकेशन उस वीडियो के उस हिस्से का लिंक भी देगा जहाँ आपके सवाल का जवाब मौजूद है।

यहाँ कुछ क्वेरीज़ दी गई हैं जिन्हें आप आज़मा सकते हैं:

- Azure Machine Learning क्या है?
- Convolutional neural networks कैसे काम करते हैं?
- Neural network क्या है?
- क्या मैं Jupyter Notebooks को Azure Machine Learning के साथ इस्तेमाल कर सकता हूँ?
- ONNX क्या है?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। यद्यपि हम सटीकता के लिए प्रयासरत हैं, कृपया ध्यान दें कि स्वचालित अनुवादों में त्रुटियाँ या गलतियाँ हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को ही प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
